In [33]:
from collections import deque
import logging
import os
import shutil
import sys

import geopandas as gpd
import numpy
import pandas
import pygeoprocessing
import pygeoprocessing.kernels
import scipy
import shapely
from osgeo import gdal
gdal.UseExceptions()

logger = logging.getLogger("pygeoprocessing")
logger.addHandler(logging.StreamHandler(stream=sys.stdout))
logger.setLevel('DEBUG')

# Preprocessing the Trucost data

This notebook just shows what pre-processing was done, but not why. A lot of effort went into figuring out the S&P Trucost dataset since it was poorly formatted and documented. For more details, see:
- https://docs.google.com/spreadsheets/d/1x5x7Ovw9tprtGWmcWgSt6bB4h0jY2O-BLFR6Y2DnMjU
- https://docs.google.com/document/d/17VCWLeJAqct9yHqKXjLqIS7SU6zd-R1OYuLVqG7yaUc

## read in data
Import dependencies and read in the original CSVs.

By default, read_csv converts certain values to numpy.nan including ‘’, ‘NA’, ‘NULL’ and others. I'm adding 'No Data' to that list because I know that it exists in some columns.

Using `dtype='string'` preserves the numeric values exactly as they are in the original file. (Instead of parsing them as floats, which can introduce error).

In [2]:
import pandas

gdrive_path = '/Users/emily/Library/CloudStorage/GoogleDrive-esoth@stanford.edu/Shared drives/MS-Planet-NatCap corporate footprinting/'

# Paths to input data in google cloud storage
asset_csv_path = f'{gdrive_path}/ES_modeling_data/source_data/Stan_Physical_AL_Expanded_LYr_20210528.csv'
owner_csv_path = f'{gdrive_path}/ES_modeling_data/source_data/Stan_PhyscialRisk_Owner_MI_LYr_20210528.csv'
ultimate_parent_csv_path = f'{gdrive_path}/ES_modeling_data/source_data/Stan_PhysicalRisk_Parent_MI_LYr_20210528.csv'

# Relative paths for outputs
asset_output_path = 'preprocessed_assets.csv'

asset_df = pandas.read_csv(asset_csv_path, dtype='string')
owner_df = pandas.read_csv(owner_csv_path, dtype='string')
ultimate_parent_df = pandas.read_csv(ultimate_parent_csv_path, dtype='string')

## standardize column name capitalization and word separators
The column names are inconsistently capitalized and some contain spaces. For consistency, make them all lowercase and replace spaces with underscores.

In [3]:
for df in [asset_df, owner_df, ultimate_parent_df]:
    # for consistency, convert all column names to lowercase
    df.rename(columns=str.lower, inplace=True)
    # replace spaces with underscores in all column names
    df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

## cross-reference the tables

There are 5 identifiers for owner and parent companies: name, MI key, CIQ ID, TC UID, and ISIN. We don't need all five in the asset table - we just need one to cross-reference with the owner ultimate parent tables, which will store the other four identifiers and info.

The MI key is defined for the most companies, so we'll keep that and drop the others. 

But first, verify that the other IDs match up when we cross-reference by the MI key.

Note: I'm not asserting that the `name`s match because I know there are inconsistencies and that's okay. We'll use the name as it appears in the owner or ultimate parent table.

In [4]:
# Assert that all 4 owner IDs match across the asset and owner tables
# The owner ISIN provided is the same as the owner ISIN derived by 
# cross-referencing with the owner table on the MI key
asset_and_owner = pandas.merge(
    left=asset_df, right=owner_df[['owner_mi_key', 'ultimate_parent_mi_key']],
    left_on='owner_mi_key', right_on='owner_mi_key', 
    suffixes=('_asset', '_owner'))

asset_and_parent = pandas.merge(
    left=asset_and_owner, right=ultimate_parent_df[['ultimate_parent_mi_key', 'ultimate_parent_isin', 'sector']],
    left_on='ultimate_parent_mi_key_owner', right_on='ultimate_parent_mi_key', 
    suffixes=('_asset', '_parent'))
assert asset_and_parent['ultimate_parent_isin_asset'].equals(asset_and_parent['ultimate_parent_isin_parent'])
assert asset_and_parent['sector_asset'].equals(asset_and_parent['sector_parent'])

In [5]:
print(sum(asset_df.duplicated()), 'duplicate rows in asset table')
asset_df = asset_df.drop_duplicates()

4058 duplicate rows in asset table


## clean up categorical data
Clean up a couple of minor issues in categorical data:

In [6]:
# facility_category: consolidate value spellings
asset_df['facility_category'] = asset_df['facility_category'].replace('Data Centre', 'Data Center')
asset_df['facility_category'] = asset_df['facility_category'].replace('Power Plants', 'Power Plant')

# data_source: fix space character
asset_df['data_source'] = asset_df['data_source'].replace(
    'European Pollutant Release\xa0and Transfer Register',
    'European Pollutant Release and Transfer Register')

# Remove trailing whitespace from activity_description values
asset_df['activity_description'] = asset_df['activity_description'].apply(lambda x: x if pandas.isna(x) else x.strip())

This wasn't documented, but it seems that the facility category and activity description go together. For most facility categories, the activity description is identical to the facility category. A few categories have their own set of activity descriptions that provide more detail about the asset.

In [7]:
facility_categories = set(asset_df['facility_category'].unique())
asset_df['activity_description'] = asset_df['activity_description'].apply(
    lambda x: None if x in facility_categories else x)

## add a unique id
The asset table has no unique identifier for each row. Create one:

In [8]:
asset_df['id'] = range(0, asset_df.shape[0])

## reorder columns
Check that the asset dataframe has all the expected columns, then organize them into an arbitrary order that I like.

In [9]:
# The 'sector' column actually contains GICS subindustries, not sectors.
# Rename it accordingly
asset_df = asset_df.rename(columns={'sector': 'gics_subindustry'})

In [10]:
asset_df = asset_df[[
    'id',
    'asset_name',
    'latitude',
    'longitude',
    'facility_category',
    'activity_description',
    'ultimate_parent_isin',
    'gics_subindustry']]

## save the preprocessed data to CSV

In [11]:
# Use empty string to represent null
asset_df.to_csv(asset_output_path, na_rep='', index=False)

# Prepare the ecosystem and biodiversity layers

In [12]:
# Update this to point to your local Google Drive
drive_path = '/Users/emily/Library/CloudStorage/GoogleDrive-esoth@stanford.edu/Shared drives/MS-Planet-NatCap corporate footprinting/'
data_dir = drive_path + 'ES_modeling_data/script data/aligned_with_PNV/'

base_endemic_biodiversity_path = data_dir + 'biodiversity_endemic_index.tif'
base_redlist_species_path = data_dir + 'biodiversity_RedList.tif'
base_species_richness_path = data_dir + 'biodiversity_species_richness_index.tif'
base_kba_path = data_dir + 'biodiversity_KBAs.tif'
base_coastal_risk_reduction_service_path = data_dir + 'coastal_risk_reduction_for_coastal_populations.tif'
base_nature_access_path = data_dir + 'nature_access_for_people.tif'
base_nitrogen_retention_service_path = data_dir + 'nitrogen_retention_for_downstream_populations.tif'
base_sediment_retention_service_path = data_dir + 'sediment_deposition_for_downstream_populations.tif'

pnv_path = data_dir + 'PNV_full_on_ESA_md5_24fe98_EckertIV_Q.tif'
pnv_nodata = pygeoprocessing.get_raster_info(pnv_path)['nodata'][0]

target_layers = {}

## Create KBA within 1 kilometer raster
The base KBA raster represents the number of KBAs covering a pixel (0, 1, or 2).
Create a KBA-within-1km raster, where 1 means there is a KBA within 1km of the pixel, and 0 means there is no KBA within 1km of the pixel.

In [13]:
warped_raster_path = 'kba_warped.tif'
kernel_path = 'kba_kernel.tif'
convolution_path = 'kba_convolved.tif'
kba_within_1km_path = 'kba_within_1km.tif'

raster_info = pygeoprocessing.get_raster_info(base_kba_path)
distance_threshold_m = 1000
pixel_size_m = abs(raster_info['pixel_size'][0])
print(pixel_size_m)

pygeoprocessing.kernels.dichotomous_kernel(
    target_kernel_path=kernel_path,
    max_distance=distance_threshold_m / pixel_size_m,
    normalize=False)

# warp with no changes, just to make the block size square before convolving
pygeoprocessing.warp_raster(
    base_raster_path=base_kba_path, 
    target_pixel_size=raster_info['pixel_size'], 
    target_raster_path=warped_raster_path,
    resample_method='near')

pygeoprocessing.convolve_2d(
    signal_path_band=(warped_raster_path, 1),
    kernel_path_band=(kernel_path, 1),
    target_path=convolution_path)

def op(block):
    result = numpy.full(block.shape, -1)
    valid_mask = ~array_equals_nodata(block, raster_info['nodata'][0])
    result[valid_mask] = block[valid_mask] > 0
    return result

pygeoprocessing.raster_calculator(
    base_raster_path_band_const_list=[(convolution_path, 1)],
    local_op=lambda array: array > 0,
    target_raster_path=kba_within_1km_path,
    datatype_target=gdal.GDT_Int16,
    nodata_target=255)

300.0
transforming bounding box from [-16920793.1795, -6951751.3822, 16920706.8205, 8375548.6178] 
transforming bounding to [-16920793.1795, -6951751.382200001, 16920706.8205, 8375548.6178] 
Warp 5.0% complete kba_warped.tif
Warp 9.0% complete kba_warped.tif
Warp 13.0% complete kba_warped.tif
Warp 18.0% complete kba_warped.tif
Warp 22.0% complete kba_warped.tif
Warp 26.0% complete kba_warped.tif
Warp 31.0% complete kba_warped.tif
Warp 35.0% complete kba_warped.tif
Warp 39.0% complete kba_warped.tif
Warp 44.0% complete kba_warped.tif
Warp 49.0% complete kba_warped.tif
Warp 53.0% complete kba_warped.tif
Warp 57.0% complete kba_warped.tif
Warp 62.0% complete kba_warped.tif
Warp 66.0% complete kba_warped.tif
Warp 70.0% complete kba_warped.tif
Warp 75.0% complete kba_warped.tif
Warp 80.0% complete kba_warped.tif
Warp 84.0% complete kba_warped.tif
Warp 88.0% complete kba_warped.tif
Warp 93.0% complete kba_warped.tif
Warp 97.0% complete kba_warped.tif
Warp 100.0% complete kba_warped.tif
start

## Set coastal risk reduction service to zero inland
The base coastal risk reduction raster has nodata in non-coastal areas. Set inland areas to zero because we know that they provide no coastal risk reduction service.

In [14]:
target_layers['coastal_risk_reduction_service'] = 'crr_masked_inland.tif'
crr_nodata = pygeoprocessing.get_raster_info(
    base_coastal_risk_reduction_service_path)['nodata'][0]

# Set CRR to 0 inland (where crr == nodata and pnv != nodata)
def crr_op(crr_array, pnv_array):
    crr_is_nodata = pygeoprocessing.array_equals_nodata(crr_array, crr_nodata)
    pnv_is_nodata = pygeoprocessing.array_equals_nodata(pnv_array, pnv_nodata)
    crr_array[crr_is_nodata & ~pnv_is_nodata] = 0
    return crr_array
pygeoprocessing.raster_calculator(
    base_raster_path_band_const_list=[
        (base_coastal_risk_reduction_service_path, 1), 
        (pnv_path, 1)], 
    local_op=crr_op, 
    target_raster_path=target_layers['coastal_risk_reduction_service'],
    datatype_target=gdal.GDT_Float32, 
    nodata_target=crr_nodata)


starting stats_worker
stats worker PID: 29224
started stats_worker <Thread(Thread-8 (stats_worker), started daemon 123145464807424)>
crr_masked_inland.tif 3.4% complete
crr_masked_inland.tif 5.5% complete
crr_masked_inland.tif 7.6% complete
crr_masked_inland.tif 9.7% complete
crr_masked_inland.tif 11.9% complete
crr_masked_inland.tif 14.0% complete
crr_masked_inland.tif 16.1% complete
crr_masked_inland.tif 18.4% complete
crr_masked_inland.tif 20.5% complete
crr_masked_inland.tif 22.6% complete
crr_masked_inland.tif 24.9% complete
crr_masked_inland.tif 27.1% complete
crr_masked_inland.tif 29.3% complete
crr_masked_inland.tif 31.5% complete
crr_masked_inland.tif 33.7% complete
crr_masked_inland.tif 36.1% complete
crr_masked_inland.tif 38.2% complete
crr_masked_inland.tif 40.4% complete
crr_masked_inland.tif 42.6% complete
crr_masked_inland.tif 44.9% complete
crr_masked_inland.tif 47.1% complete
crr_masked_inland.tif 49.4% complete
crr_masked_inland.tif 51.7% complete
crr_masked_inland.ti

## Set biodiversity layers to nodata in the oceans
The base biodiversity layers have 0 in the oceans. Change to nodata, because the biodiversity metrics don't apply to oceans, and the large area of 0 would throw off the percentile calculation.

The base ecosystem service layers already have nodata in the oceans. 

In [15]:
# Set biodiversity layers to nodata in the oceans (where biodiversity layer == 0 and pnv == nodata)
for key, base_path in [
        ('endemic_biodiversity', base_endemic_biodiversity_path),
        ('kba_within_1km', kba_within_1km_path),
        ('redlist_species',  base_redlist_species_path),
        ('species_richness', base_species_richness_path)]:
    es_raster_info = pygeoprocessing.get_raster_info(base_path)
    es_nodata = es_raster_info['nodata'][0]
    es_dtype = es_raster_info['datatype']
    target_layers[key] = f'masked_{key}.tif'
    
    def mask(es_array, pnv_array):
        pnv_is_nodata = pygeoprocessing.array_equals_nodata(pnv_array, pnv_nodata)
        es_array[(es_array == 0) & pnv_is_nodata] = es_nodata
        return es_array
        
    pygeoprocessing.raster_calculator(
        base_raster_path_band_const_list=[(base_path, 1), (pnv_path, 1)],
        local_op=mask, 
        target_raster_path=target_layers[key], 
        datatype_target=es_dtype, 
        nodata_target=es_nodata)
    

starting stats_worker
stats worker PID: 29224
started stats_worker <Thread(Thread-9 (stats_worker), started daemon 123145464807424)>
masked_endemic_biodiversity.tif 5.5% complete
masked_endemic_biodiversity.tif 8.7% complete
masked_endemic_biodiversity.tif 11.6% complete
masked_endemic_biodiversity.tif 14.7% complete
masked_endemic_biodiversity.tif 17.5% complete
masked_endemic_biodiversity.tif 20.4% complete
masked_endemic_biodiversity.tif 23.2% complete
masked_endemic_biodiversity.tif 26.0% complete
masked_endemic_biodiversity.tif 28.9% complete
masked_endemic_biodiversity.tif 31.7% complete
masked_endemic_biodiversity.tif 34.5% complete
masked_endemic_biodiversity.tif 37.4% complete
masked_endemic_biodiversity.tif 40.5% complete
masked_endemic_biodiversity.tif 43.3% complete
masked_endemic_biodiversity.tif 46.0% complete
masked_endemic_biodiversity.tif 48.8% complete
masked_endemic_biodiversity.tif 51.4% complete
masked_endemic_biodiversity.tif 54.1% complete
masked_endemic_biodiver

In [16]:
import pprint
pp = pprint.PrettyPrinter()

target_layers['sediment_retention_service'] = base_sediment_retention_service_path
target_layers['nitrogen_retention_service'] = base_nitrogen_retention_service_path
target_layers['nature_access'] = base_nature_access_path
pp.pprint(target_layers)

{'coastal_risk_reduction_service': 'crr_masked_inland.tif',
 'endemic_biodiversity': 'masked_endemic_biodiversity.tif',
 'kba_within_1km': 'masked_kba_within_1km.tif',
 'nature_access': '/Users/emily/Library/CloudStorage/GoogleDrive-esoth@stanford.edu/Shared '
                  'drives/MS-Planet-NatCap corporate '
                  'footprinting/ES_modeling_data/script '
                  'data/aligned_with_PNV/nature_access_for_people.tif',
 'nitrogen_retention_service': '/Users/emily/Library/CloudStorage/GoogleDrive-esoth@stanford.edu/Shared '
                               'drives/MS-Planet-NatCap corporate '
                               'footprinting/ES_modeling_data/script '
                               'data/aligned_with_PNV/nitrogen_retention_for_downstream_populations.tif',
 'redlist_species': 'masked_redlist_species.tif',
 'sediment_retention_service': '/Users/emily/Library/CloudStorage/GoogleDrive-esoth@stanford.edu/Shared '
                               'drives/MS-Plane

# Prepare the ES layer table for the footprinting tool

## Calculate the 90th percentile of each layer

## Format the table

In [18]:
pandas.options.display.float_format = '{:.15f}'.format
es_ids = []
paths = []
thresholds = []
for key, path in target_layers.items():
    es_ids.append(key)
    paths.append(path)
    thresholds.append(percentile_df[90][key])
df = pandas.DataFrame({
    'es_id': es_ids,
    'es_value_path': paths,
    'flag_threshold': thresholds
}).set_index('es_id')
df.to_csv('es_layer_table.csv', float_format='{:.15f}')
df

,es_value_path,flag_threshold
es_id,,
coastal_risk_reduction_service,crr_masked_inland.tif,0.000000000000000
endemic_biodiversity,masked_endemic_biodiversity.tif,0.000005344048077
kba_within_1km,masked_kba_within_1km.tif,0.000000000000000
redlist_species,masked_redlist_species.tif,19.000000000000000
species_richness,masked_species_richness.tif,0.067348398268223
sediment_retention_service,/Users/emily/Library/CloudStorage/GoogleDrive-...,122161.820312500000000
nitrogen_retention_service,/Users/emily/Library/CloudStorage/GoogleDrive-...,37028224.000000000000000
nature_access,/Users/emily/Library/CloudStorage/GoogleDrive-...,70930.000000000000000


# Prepare point and linear asset data

## Filter to assets owned by companies in MSCI ACWI

In [20]:
# Companies may have multiple ISIN numbers representing different securities
# Filter to just those assets whose 'ultimate_parent_isin' is in the MSCI ACWI table 'ISIN' column
msci_acwi_df = pandas.read_csv(f'{gdrive_path}/MSCI/msci_acwi_expanded_20221231.csv')
msci_acwi_isins = set(msci_acwi_df['ISIN'].unique())

print('read')
asset_df = pandas.read_csv('preprocessed_assets.csv')

print('apply')
asset_df = asset_df[asset_df['ultimate_parent_isin'].apply(lambda isin: isin in msci_acwi_isins)]

read
apply


## Separate out linear assets and point assets
Natural gas pipelines and transmission lines are represented by multiple asset points, which we need to group together and connect to form multilinestrings.

All the other asset types are represented by just one point.

In [21]:
linear_asset_df = asset_df[asset_df['facility_category'].apply(  # filter to linear asset types
    lambda val: val in {'Natural Gas Pipeline', 'Transmission Line'})].copy()
point_asset_df = asset_df[asset_df['facility_category'].apply(  # filter to non-linear asset types
    lambda val: val not in {'Natural Gas Pipeline', 'Transmission Line'})].copy()

# write out point assets
point_asset_df.to_csv('msci_acwi_point_assets.csv', index=False)

## Group linear asset points together into multilinestrings

In [25]:
def multilinestring_traversal(graph):
    """
    Traverse a connected, undirected tree graph to produce a multilinestring representing the tree shape.
    
    Similar to pre-order depth-first traversal, but revisits branch points.
    
    Example:
    
    [[0 1 0 0 0 0 0]              0
     [1 0 1 0 1 0 0]              |
     [0 1 0 1 0 0 0]              1
     [0 0 1 0 0 0 0]     =       / \      ->  [[0, 1, 2, 3], [1, 4, 5], [4, 6]]
     [0 1 0 0 0 1 1]            2   4
     [0 0 0 0 1 0 0]           /   / \
     [0 0 0 0 1 0 0]]         3   5   6
        

    Args:
        graph (numpy.array): NxN adjacency matrix representing the graph
        
    Returns:
        list[list[int]]. List (representing a multilinestring) of lists (representing linestrings) of 
        integer indices (0..(N-1))
    """
    # make a copy because we'll modify it
    graph = numpy.copy(graph)
    
    # start on an arbitrary leaf node        
    for i, row in enumerate(graph):
        if row.sum() == 1:
            current_node = i
            break
    
    branch_points = deque([current_node])
    linestrings = []

    # while there are branches we haven't traversed yet
    while branch_points:
         
        linestring = []  # start a new linestring representing this branch
        current_node = branch_points.pop()
        next_available_nodes = set([current_node])
        
        # if at least one neighbor is available, continue along this branch
        while next_available_nodes:

            # if more than one neighbor is available, store this node to revisit later
            if len(next_available_nodes) > 1:
                branch_points.append(current_node)
                
            current_node = next_available_nodes.pop()
            
            linestring.append(current_node)
            
            # mark this node as visited
            # after this, it won't be available as a neighbor of any other node
            graph[:, current_node] = False
            
            # get the set of nodes that are adjacent to current_node and not yet visited
            next_available_nodes = set(list(numpy.argwhere(graph[current_node]).flatten()))
        
        linestrings.append(linestring)

    return linestrings


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/var/folders/zh/dy02ljyd4mj5_nfm158hwcr00000gn/T/ipykernel_29224/4023453138.py:2: SyntaxWarning: invalid escape sequence '\ '
  """


In [34]:
# Group points together by facility category
for category, category_group in linear_asset_df.groupby('facility_category'):
    print(category)

    names, geoms = [], []

    # Group points together by their asset_name attribute
    for name, group in category_group.groupby('asset_name'):
        print(name, group.shape[0])
        if group.shape[0] == 1:
            print(f'Warning: group {name} has only one point, skipping')
            continue

        # convert to a projected coordinate system for accurate distance calculations
        gs = gpd.GeoSeries(
            gpd.points_from_xy(group.longitude, group.latitude, crs='EPSG:4326')
        ).to_crs('ESRI:54012')

        # calculate distance matrix: distance from each point to each other point
        distance_matrix = gs.apply(lambda point: gs.distance(point))

        # calculate minimum spanning tree
        mst = scipy.sparse.csgraph.minimum_spanning_tree(distance_matrix).toarray()

        # convert to an undirected, binary adjacency matrix
        mst = (mst + mst.T).astype(bool)

        multilinestring = multilinestring_traversal(mst)

        # convert point indices to point coordinates
        multilinestring = shapely.MultiLineString([
            shapely.LineString([
                shapely.Point(gs[idx].x, gs[idx].y) for idx in linestring
            ]) for linestring in multilinestring])

        names.append(name)
        geoms.append(multilinestring)

    new_gdf = gpd.GeoDataFrame(
        pd.DataFrame({'name': names}), 
        geometry=gpd.GeoSeries(geoms), 
        crs='ESRI:54012')
    new_gdf.to_file('msci_acwi_linear_asset_multilinestrings.gpkg', layer=category)

Natural Gas Pipeline
300 Line Expansion 321
85 North Expansion 57
ANR Pipeline 15890
ANR Storage Company 45
Access South 25
Adair Southwest 13
Agua Blanca Pipeline 963
Aitken Creek Looping 23
Algonquin Gas Transmission 2853
Algonquin Incremental Market (AIM) 97
Anchor Point Pipeline 57
Apex Expansion 72
Appalachian Gateway 275
Arkoma Connector Pipeline 138
Ashland Pipeline 30
Atlantic Bridge 36
Atlantic Sunrise Expansion/Central Penn North 148
Atlantic Sunrise Expansion/Central Penn South 329
Atmos Pipeline-Texas 19607
Auburn Gathering System Phase 1 26
Auburn Gathering System Phase 2 72
Auburn Pipeline Extension 26
BC Pipeline 4576
Big Pine Expansion 24
Big Pine Gathering System 144
Big Sandy Pipeline 174
Birdsboro Pipeline 36
Bison Pipeline 771
Bissette Pipeline 45
Black Marlin Pipeline 226
Blanco-Meeker Expansion 786
Blue Union Gathering System 688
Bluebonnet Market Express Pipeline 1184
Bluestone Gathering Pipeline 119
Bluewater Gas Storage Pipeline 102
Bobcat Storage 47
Brazoria I

KeyboardInterrupt: 

## Manually clean up the linear assets

In [ ]:
# Jesse cleaned up the linear assets generated in the previous step
cleaned_linear_assets_path = f'{gdrive_path}/ES_modeling_data/asset_footprints/linear_assets_cleaned.gpkg'

## Buffer linear assets to create polygons

In [ ]:
transmission_line_gdf = gpd.read_file(
    cleaned_linear_assets_path, 
    layer='transmission_lines_clean')
print(transmission_line_gdf)
# cap_style=2: flat end caps (buffer does not extend past the end of each line)
transmission_line_gdf.geometry = transmission_line_gdf.geometry.buffer(42.860707, cap_style=2)
transmission_line_gdf['facility_category'] = 'Transmission Line'

print(transmission_line_gdf)

pipeline_gdf = gpd.read_file(
    cleaned_linear_assets_path, 
    layer='pipelines_clean')
print(pipeline_gdf)
pipeline_gdf.geometry = pipeline_gdf.geometry.buffer(23.9395345, cap_style=2)
pipeline_gdf['facility_category'] = 'Natural Gas Pipeline'
print(pipeline_gdf)
gdf = pd.concat([pipeline_gdf, transmission_line_gdf]) 
gdf

In [12]:
gdf.to_file('linear_assets_buffered.gpkg', layer='assets')

# Run the footprinting tool

## Merge ultimate parent data back into linear assets

In [ ]:
linear_asset_stats_df = pd.read_csv('/Users/emily/natural-capital-footprint-impact/out_linear_assets.csv')
asset_name_isins = linear_asset_df.groupby(['asset_name', 'ultimate_parent_isin']).count().reset_index()[['asset_name', 'ultimate_parent_isin']]
linear_asset_stats_df = pd.merge(
    left=linear_asset_stats_df,
    right=asset_name_isins,
    left_on='name',
    right_on='asset_name',
    how='inner').drop('name', axis='columns')

## Consolidate ISINs

In [ ]:
# Create a mapping of expanded to original ISINs,
# excluding those without a clear mapping
expanded_acwi_df = pd.read_csv('msci_acwi_expanded_20221231.csv')
print(len(expanded_acwi_df['ISIN_original'].unique()))
counts = expanded_acwi_df['ISIN'].value_counts()
duplicated_isins = set(counts[counts > 1].index)

isin_is_duplicated = expanded_acwi_df[expanded_acwi_df['ISIN'].apply(
    lambda isin: isin in duplicated_isins)]

expanded_acwi_df = expanded_acwi_df.drop(isin_is_duplicated.index)
print(len(expanded_acwi_df['ISIN_original'].unique()))

expanded_to_original_isin = {}
for _, row in expanded_acwi_df.iterrows():
    expanded_to_original_isin[row['ISIN']] = row['ISIN_original']
    
original_acwi_df = pd.read_csv('/Users/emily/Downloads/msci_acwi_20221231.csv')
all(original_acwi_df['ISIN'] == original_acwi_df['ISIN_original'])

In [ ]:
asset_df['ultimate_parent_isin'] = asset_df['ultimate_parent_isin'].apply(
    lambda isin: expanded_to_original_isin[isin])
len(asset_df['ultimate_parent_isin'].unique())

In [ ]:
linear_asset_stats_df['ultimate_parent_isin'] = linear_asset_stats_df['ultimate_parent_isin'].apply(
    lambda isin: expanded_to_original_isin[isin])
point_asset_stats_df = pd.read_csv('/Users/emily/natural-capital-footprint-impact/out.csv', low_memory=False)
point_asset_stats_df['ultimate_parent_isin'] = point_asset_stats_df['ultimate_parent_isin'].apply(
    lambda isin: expanded_to_original_isin[isin])
asset_stats_df = pd.concat([point_asset_stats_df, linear_asset_stats_df], ignore_index=True)
original_isins = set(expanded_acwi_df['ISIN_original'].unique())
asset_stats_df['ultimate_parent_isin'].apply(lambda isin: isin not in original_isins).sum()

## Calculate adjusted sum

In [ ]:
es_ids = [
    'coastal_risk_reduction_service',
    'nitrogen_retention_service',
    'sediment_retention_service',
    'nature_access',
    'endemic_biodiversity',
    'redlist_species',
    'species_richness',
    'kba_within_1km']

In [ ]:
asset_stats_df['footprint_area'] = gpd.GeoSeries.from_wkt(
    asset_stats_df['WKT'], crs='ESRI:54012').area

In [ ]:
for es_id in es_ids:
    asset_stats_df = asset_stats_df.drop(f'{es_id}_sum', axis=1)
    if es_id == 'nature_access':
        pixel_area = 2000**2
    else:
        pixel_area = 300**2
    asset_stats_df[f'{es_id}_adj_sum'] = (
        asset_stats_df[f'{es_id}_mean'] * asset_stats_df['footprint_area'] / pixel_area)

In [ ]:
asset_stats_df.to_csv('msci_acwi_asset_stats.csv', index=False)

## Create company results table

In [ ]:
from impact.src import aggregate_footprints

gs = gpd.GeoSeries.from_wkt(asset_stats_df['WKT'])
asset_stats_gdf = gpd.GeoDataFrame(asset_stats_df, geometry=gs, crs="ESRI:54012")

aggregate_footprints(
    asset_stats_gdf, 
    'msci_acwi_company_results.csv',
    'ultimate_parent_isin',
    'polygons')

In [ ]:
company_stats_df = pd.read_csv('msci_acwi_company_results.csv')
company_stats_df = company_stats_df.rename(
    columns={'ultimate_parent_isin': 'ISIN'}
).set_index('ISIN')
company_stats_df['total_flagged_no_nature_access'] = None

for isin, row in company_stats_df.iterrows():
    
    company_rows = asset_stats_gdf[
        asset_stats_gdf['ultimate_parent_isin'] == isin].copy()
    company_rows['any_flag_no_nature_access'] = [False for _ in range(company_rows.shape[0])]

    for es_id in es_ids:
        if es_id != 'nature_access':
            company_rows['any_flag_no_nature_access'] = (
                company_rows['any_flag_no_nature_access'] | company_rows[f'{es_id}_flag'])

    company_stats_df.loc[isin, 'total_flagged_no_nature_access'] = (
        company_rows['any_flag_no_nature_access'].sum())

company_stats_df['total_flagged_no_nature_access'] = (
    company_stats_df['total_flagged_no_nature_access'].astype(int))
company_stats_df

In [ ]:
original_acwi_df = pd.read_csv(
    '/Users/emily/Downloads/msci_acwi_20221231.csv',
    index_col='ISIN',
    usecols=[
        'ISIN',
        'Company Name',
        'Country Code',
        'Region',
        'GICS Sector',
        'GICS Industry Group',
        'GICS Industry',
        'GICS Sub-Industry',
        'MKTCAP_USD',
        'PRICE_TO_SALES',
        'SALES_USD'])
results_df = pd.merge(
    left=company_stats_df,
    right=original_acwi_df,
    left_index=True,
    right_index=True)

In [ ]:
results_df.to_csv('msci_acwi_company_results.csv', index_label='ISIN')

In [ ]:
old_results_df = pd.read_csv('/Users/emily/Downloads/msci_acwi_company_results.csv', index_col='ISIN')
old_results_df

In [ ]:
(
    results_df['nature_access_flagged'].sort_index() != 
    old_results_df['nature_access_flags']).any()

In [ ]:
(results_df['total_flagged'].sort_index() - old_results_df['total_flags']).sort_values()